In [1]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR

## Loading and Preprocessing Loans Encoded Data

In [2]:
# Loading data
file_path = Path("resources/cleaned_close1.csv")
df_equities = pd.read_csv(file_path)
df_equities.head()

,symbol,date,high,iexClose,industry,low,sector,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,p_status
0,PJUL,8/19/2020,27.84,27.710,Investment Trusts/Mutual Funds,27.76,Miscellaneous,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
1,PIM,8/19/2020,4.26,4.240,Investment Trusts/Mutual Funds,4.23,Miscellaneous,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0
2,PJT,8/19/2020,59.67,58.885,Investment Banks/Brokers,58.60,Finance,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
3,PKBK,8/19/2020,13.46,13.460,Regional Banks,12.89,Finance,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
4,PICK,8/19/2020,28.49,28.380,Investment Trusts/Mutual Funds,28.25,Miscellaneous,71208,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,Null


In [3]:
df_equities.dtypes

symbol                       object
date                         object
high                        float64
iexClose                    float64
industry                     object
low                         float64
sector                       object
volume                        int64
death                         int64
deathIncrease                 int64
hospitalizedIncrease          int64
hospitalizedCurrently         int64
negative                      int64
negativeIncrease              int64
positive                      int64
positiveIncrease              int64
totalTestResults              int64
totalTestResultsIncrease      int64
p_status                     object
dtype: object

In [4]:
# Drop null rows
df_equities = df_equities.dropna()
df_equities.drop(df_equities.loc[df_equities['p_status']== "Null"].index, inplace=True)

In [5]:
df_equities.head(20)

,symbol,date,high,iexClose,industry,low,sector,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,p_status
0,PJUL,8/19/2020,27.840,27.710,Investment Trusts/Mutual Funds,27.760,Miscellaneous,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
1,PIM,8/19/2020,4.260,4.240,Investment Trusts/Mutual Funds,4.230,Miscellaneous,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0
2,PJT,8/19/2020,59.670,58.885,Investment Banks/Brokers,58.600,Finance,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
3,PKBK,8/19/2020,13.460,13.460,Regional Banks,12.890,Finance,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
5,PK,8/19/2020,8.770,8.605,Real Estate Investment Trusts,8.440,Finance,2894906,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
6,PJP,8/19/2020,66.675,66.420,Investment Trusts/Mutual Funds,66.260,Miscellaneous,14548,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
7,PLBC,8/19/2020,20.830,20.895,Regional Banks,20.620,Finance,616,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
8,PKE,8/19/2020,11.480,11.420,Aerospace & Defense,11.290,Electronic Technology,43723,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0
9,PJUN,8/19/2020,29.250,28.980,Investment Trusts/Mutual Funds,29.105,Miscellaneous,1365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1
10,PLAB,8/19/2020,11.380,11.320,Electronic Production Equipment,11.240,Electronic Technology,95263,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0


In [6]:
float or np.ndarray(dtype=float)
# df = pd.read_csv("resources/cleaned_close.csv")
df_equities['date'] = pd.to_datetime(df_equities['date'])    
df_equities['date_delta'] = (df_equities['date'] - df_equities['date'].min())  / np.timedelta64(1,'D')

In [7]:
df_equities.dtypes

symbol                              object
date                        datetime64[ns]
high                               float64
iexClose                           float64
industry                            object
low                                float64
sector                              object
volume                               int64
death                                int64
deathIncrease                        int64
hospitalizedIncrease                 int64
hospitalizedCurrently                int64
negative                             int64
negativeIncrease                     int64
positive                             int64
positiveIncrease                     int64
totalTestResults                     int64
totalTestResultsIncrease             int64
p_status                            object
date_delta                         float64
dtype: object

In [8]:
# converting to int64
df_equities = df_equities.astype({"date":'int64'}) 

C:\Users\japodaca\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


In [9]:
df_equities.dtypes

symbol                       object
date                          int64
high                        float64
iexClose                    float64
industry                     object
low                         float64
sector                       object
volume                        int64
death                         int64
deathIncrease                 int64
hospitalizedIncrease          int64
hospitalizedCurrently         int64
negative                      int64
negativeIncrease              int64
positive                      int64
positiveIncrease              int64
totalTestResults              int64
totalTestResultsIncrease      int64
p_status                     object
date_delta                  float64
dtype: object

In [10]:
df_equities.head()

,symbol,date,high,iexClose,industry,low,sector,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,p_status,date_delta
0,PJUL,1597795200000000000,27.84,27.710,Investment Trusts/Mutual Funds,27.76,Miscellaneous,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
1,PIM,1597795200000000000,4.26,4.240,Investment Trusts/Mutual Funds,4.23,Miscellaneous,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0,0.0
2,PJT,1597795200000000000,59.67,58.885,Investment Banks/Brokers,58.60,Finance,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
3,PKBK,1597795200000000000,13.46,13.460,Regional Banks,12.89,Finance,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
5,PK,1597795200000000000,8.77,8.605,Real Estate Investment Trusts,8.44,Finance,2894906,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0


In [11]:
df= df_equities.drop(columns = ["symbol", "industry", "sector"])

In [12]:
df.head()

,date,high,iexClose,low,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,p_status,date_delta
0,1597795200000000000,27.84,27.710,27.76,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
1,1597795200000000000,4.26,4.240,4.23,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0,0.0
2,1597795200000000000,59.67,58.885,58.60,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
3,1597795200000000000,13.46,13.460,12.89,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0
5,1597795200000000000,8.77,8.605,8.44,2894906,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,1,0.0


In [13]:
# Define features set
X = df_equities.copy()
X = X.drop("p_status", axis=1)
X.head()

,symbol,date,high,iexClose,industry,low,sector,volume,death,deathIncrease,hospitalizedIncrease,hospitalizedCurrently,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease,date_delta
0,PJUL,1597795200000000000,27.84,27.710,Investment Trusts/Mutual Funds,27.76,Miscellaneous,3283,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0.0
1,PIM,1597795200000000000,4.26,4.240,Investment Trusts/Mutual Funds,4.23,Miscellaneous,31824,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0.0
2,PJT,1597795200000000000,59.67,58.885,Investment Banks/Brokers,58.60,Finance,23896,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0.0
3,PKBK,1597795200000000000,13.46,13.460,Regional Banks,12.89,Finance,2365,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0.0
5,PK,1597795200000000000,8.77,8.605,Real Estate Investment Trusts,8.44,Finance,2894906,165088,1411,1983,43406,24462674,243232,5485765,45073,76356767,832115,0.0


In [14]:
# Define target vector
y = df_equities["p_status"].values.reshape(-1, 1)
y[:5]

array([['1'],
       ['0'],
       ['1'],
       ['1'],
       ['1']], dtype=object)

In [15]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(427800, 19)
(142600, 19)
(427800, 1)
(142600, 1)


In [17]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)

In [18]:
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(456320, 19)
(114080, 19)
(456320, 1)
(114080, 1)


In [19]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [20]:
df_equities.dtypes

symbol                       object
date                          int64
high                        float64
iexClose                    float64
industry                     object
low                         float64
sector                       object
volume                        int64
death                         int64
deathIncrease                 int64
hospitalizedIncrease          int64
hospitalizedCurrently         int64
negative                      int64
negativeIncrease              int64
positive                      int64
positiveIncrease              int64
totalTestResults              int64
totalTestResultsIncrease      int64
p_status                     object
date_delta                  float64
dtype: object

In [21]:
# df_equities[df_equities].nunique()


In [23]:
df_equities.dtypes

symbol                       object
date                          int64
high                        float64
iexClose                    float64
industry                     object
low                         float64
sector                       object
volume                        int64
death                         int64
deathIncrease                 int64
hospitalizedIncrease          int64
hospitalizedCurrently         int64
negative                      int64
negativeIncrease              int64
positive                      int64
positiveIncrease              int64
totalTestResults              int64
totalTestResultsIncrease      int64
p_status                     object
date_delta                  float64
dtype: object

In [24]:
df_equities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570400 entries, 0 to 631359
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   symbol                    570400 non-null  object 
 1   date                      570400 non-null  int64  
 2   high                      570400 non-null  float64
 3   iexClose                  570400 non-null  float64
 4   industry                  570400 non-null  object 
 5   low                       570400 non-null  float64
 6   sector                    570400 non-null  object 
 7   volume                    570400 non-null  int64  
 8   death                     570400 non-null  int64  
 9   deathIncrease             570400 non-null  int64  
 10  hospitalizedIncrease      570400 non-null  int64  
 11  hospitalizedCurrently     570400 non-null  int64  
 12  negative                  570400 non-null  int64  
 13  negativeIncrease          570400 non-null  i

In [25]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

ValueError: could not convert string to float: 'DDWM'

In [ ]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Fitting the Decision Tree Model

In [ ]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [ ]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

## Making Predictions Using the Tree Model

In [ ]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)
predictions

## Model Evaluation

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))
